In [1]:
import re
from urllib import error
from html import unescape
import requests
import json 
from bs4 import BeautifulSoup
import time


In [28]:
import sqlite3

In [2]:
def download(method,url,param=None,data=None, timeout=1, maxretries=3):
    headers ={"Accept-Language": "ko-KR",
"Client-Id": "kimne78kx3ncx6brgo4mv6wki5h1ko",
"Content-Type": "text/plain;charset=UTF-8",
"Origin": "https://www.twitch.tv",
"Referer": "https://www.twitch.tv/search?term=%ED%92%8D%EC%9B%94%EB%9F%89%2F459527458",
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36",
"X-Device-Id": "1896bda3b7a429f1"}
#  개발자 도구에서 qql의 request헤더부분 설정함
    try:
        resp=requests.request(method, url,params=param,data=data, headers=headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500<=e.response.status_code<600 and maxretries>0:
            print(maxretries)
            
            download(method,url,param,data,timeout,maxretries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [29]:
con = sqlite3.connect("chats.db")
cur = con.cursor()
cur.executescript('''
    DROP TABLE IF EXISTS url;
    CREATE TABLE url(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    netloc_id INTEGER NOT NULL,
    path TEXT NOT NULL,
    param TEXT,
    bj_id INTEGER NOT NULL,
    title TEXT NOT NULL
    );

    DROP TABLE IF EXISTS netloc;
    CREATE TABLE netloc(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    netloc TEXT NOT NULL
    );

    DROP TABLE IF EXISTS chat;
    CREATE TABLE chat(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    content TEXT NOT NULL,
    writer_id INTEGER NOT NULL,
    w_time TEXT NOT NULL, 
    url_id INTEGER NOT NULL
    );

    DROP TABLE IF EXISTS jamak;
    CREATE TABLE jamak(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    content TEXT NOT NULL,
    j_time TEXT NOT NULL,
    url_id INTEGER NOT NULL
    );

    DROP TABLE IF EXISTS bj;
    CREATE TABLE bj(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    name TEXT NOT NULL,
    netloc_id INTEGER NOT NULL
    );

    DROP TABLE IF EXISTS writer;
    CREATE TABLE writer(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    name TEXT NOT NULL
    );
    
    ''')

con.commit()

In [6]:
# 개발자 도구에서 qql의 request payload설정부분
def search_twi(name_bj):
    return {"operationName":"SearchResultsPage_SearchResults","variables":{"query":name_bj,"options":None},"extensions":{"persistedQuery":{"version":1,"sha256Hash":"e5763309c0c0d14e7b59d5da0975794469cab4182e85b8f220820a43ff15905c"}}}
    

In [8]:
def getIdOfVideos(name_bj):
    basic_url = "https://api.twitch.tv/v5/videos/id_part/comments?content_offset_seconds=0" 
    #모든 영상 첫부분이 이 형태이고 id_part만 다르기 때문에 이렇게 성정하고 id_part부분만 수정할 것
    data = search_twi(name_bj)
    #post를 보내기위해 payload를 불러온다.
    data = json.dumps(data)
    #json형식이 object안에 object가 있는 형태라면 dumps를 써야한다. 이유는 모른다.
    html = download("post","https://gql.twitch.tv/gql",data = data)
    #qql에 post를 보내는 부분     
    if "items" in html.json()['data']['searchFor']["videos"]:
#        해당 페이지내에 비디오가 있는 지 확인후
        list_id =[_['id'] for _ in html.json()['data']['searchFor']["videos"]['items']]
#      basic_url의 id part부분에 사용할 비디오의 id들을 리스트로 받아온다
        return [basic_url.replace(requests.compat.urlparse(basic_url)[2].split('/')[3],_) for _ in list_id]
#      basic_url의 id part부분을 받아온 비디오의 id로 교체한다.
    else :
        return []
#     아니면 빈 리스트 반환

In [30]:
chat_list = []
#채팅리스트 초기값은 빈리스티이다.
def twi_chat(url):
    html = download("get",url)
    twi_json = html.json()
    chat_list.extend([(comment['commenter']['name'],comment['message']['fragments'][0]['text'] ,comment['updated_at'])for comment in twi_json["comments"]])
    if "_next" in twi_json:
        print("----동작중----")
        temp = twi_json["_next"]
        url = url.replace(requests.compat.urlparse(url)[4],'cursor='+temp)
        twi_chat(url)
    else :
        return chat_list

In [14]:
def get_url(lst):
    return list(map(getIdOfVideos,lst))

In [15]:
lst = ["이선생","앰비션","풍월량"]
get_url(lst)

[['https://api.twitch.tv/v5/videos/210486729/comments?content_offset_seconds=0',
  'https://api.twitch.tv/v5/videos/460817855/comments?content_offset_seconds=0',
  'https://api.twitch.tv/v5/videos/460475253/comments?content_offset_seconds=0',
  'https://api.twitch.tv/v5/videos/460008316/comments?content_offset_seconds=0',
  'https://api.twitch.tv/v5/videos/459532825/comments?content_offset_seconds=0',
  'https://api.twitch.tv/v5/videos/459079233/comments?content_offset_seconds=0',
  'https://api.twitch.tv/v5/videos/458658180/comments?content_offset_seconds=0',
  'https://api.twitch.tv/v5/videos/458216116/comments?content_offset_seconds=0',
  'https://api.twitch.tv/v5/videos/457800532/comments?content_offset_seconds=0',
  'https://api.twitch.tv/v5/videos/457782961/comments?content_offset_seconds=0'],
 ['https://api.twitch.tv/v5/videos/458272189/comments?content_offset_seconds=0',
  'https://api.twitch.tv/v5/videos/458272119/comments?content_offset_seconds=0',
  'https://api.twitch.tv/v5

In [31]:
twi_chat('https://api.twitch.tv/v5/videos/458216116/comments?content_offset_seconds=0')

----동작중----
----동작중----
----동작중----
----동작중----
----동작중----
----동작중----
----동작중----
----동작중----
----동작중----
----동작중----
----동작중----
----동작중----
----동작중----
----동작중----


In [32]:
chat_list

[('arm9028', '센세 굿모닝', '2019-07-26T13:46:13.998Z'),
 ('evenday77', '와 당대형이랑 같이 사는거야?', '2019-07-26T13:46:55.232Z'),
 ('ehgns12123', '좋은아침입니다', '2019-07-26T13:47:04.488Z'),
 ('arm9028', '디코', '2019-07-26T13:47:08.059Z'),
 ('habata07', '센하', '2019-07-26T13:47:14.996Z'),
 ('habata07', 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '2019-07-26T13:47:20.825Z'),
 ('arm9028', '오늘 끝까지 가나요', '2019-07-26T13:47:29.828Z'),
 ('habata07', '아 맞네 전자담배 누워서 담배 필수 잇구나.', '2019-07-26T13:47:38.904Z'),
 ('habata07', '재가 안 떨어지니끼 ㅋㅋㅋㅋ', '2019-07-26T13:47:47.272Z'),
 ('emto0103', '와 울펜!', '2019-07-26T13:47:54.69Z'),
 ('emto0103', '와 당대형!', '2019-07-26T13:47:58.862Z'),
 ('emto0103', '무개념이네 ㄷㄷ', '2019-07-26T13:48:38.627Z'),
 ('habata07', '에어컨 켜도 찝찝하긴 하겟다.', '2019-07-26T13:48:59.204Z'),
 ('arm9028', '원래 내리고 타지 않나', '2019-07-26T13:49:01.44Z'),
 ('habata07', '사람 많으면 ㅋㅋㅋㅋ', '2019-07-26T13:49:04.038Z'),
 ('arm9028', '9호선', '2019-07-26T13:49:10.949Z'),
 ('habata07', 'ㅋㅋㅋㅋㅋㅋ', '2019-07-26T13:49:28.928Z'),
 ('ssakdook', '타 게임, 타 스트리머, 성별 및 정치 발언 자제 바랍니다